### [전국 대학 기준 학과경쟁력분석 지표]
    전국 데이터를 기준으로 학과경쟁력분석 지표 산출하는 코드

---
- 신입생 충원율 및 경쟁률
- 중도탈락률
- 졸업생 취업률 및 진학률
- 전임교원 강의 비율
- 캡스톤디자인 이수 학생 수
- 교원 1인당 연구 실적
- 교원 1인당 연구비 수혜 실적
- 재학생 충원율
- 최종 병합
- 엑셀 파일 저장


### 함수 정의

In [1]:
#엑셀 파일 불러오는 함수
import pandas as pd

def load_and_clean_data(filepath, header):
    df = pd.read_excel(filepath, header = header)

    new_columns = []
    for col in df.columns:
        merged_name = [str(c) for c in col if 'Unnamed' not in str(c)]
        new_columns.append(' '.join(merged_name).strip())

    df.columns = new_columns
    df= df.fillna(method='ffill', axis=0)

    return df

In [2]:
# 특수기호 제거 + 공백 제거 함수 / 텍스트 정리
def clean_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = df[col].str.replace('·', '', regex=False)   # U+00B7
        df[col] = df[col].str.replace('ㆍ', '', regex=False)  # U+318D
        df[col] = df[col].str.replace('・', '', regex=False)  # U+30FB
        df[col] = df[col].str.replace('?', '', regex=False)
        df[col] = df[col].str.replace(r'\s*_(제\d+캠퍼스|분교)', r'_\1', regex=True)
        df[col] = df[col].str.strip()
    return df

### 신입생 충원율 및 경쟁률

In [3]:
df_freshman = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\신입생충원율,신입생경쟁률\2023년 _대학_4-다. 신입생 충원 현황_학과별자료.xlsx", header=[3,4,5])

#기준연도 정수로 변경
df_freshman['기준연도'] = df_freshman['기준연도'].astype(int)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [4]:
df_freshman.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(모집단위),구분,학과특성,...,지원자 계,지원자 정원내\n(C),지원자 정원외,입학자 계,입학자 정원내(D) 남,입학자 정원내(D) 여,입학자 정원외 남,입학자 정원외 여,정원내\n신입생 충원율(%)\n(D/B) × 100,경쟁률\n(C/B)
0,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,1665,1486,179,186,41,124,5,16,100.0,9.0
1,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,670,658,12,56,37,16,2,1,100.0,12.4
2,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,주간,일반과정,...,338,338,0,46,35,11,0,0,100.0,7.3
3,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,사회복지상담학과,주간,일반과정,...,114,114,0,13,7,6,0,0,50.0,4.4
4,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,스포츠재활복지학부,주간,일반과정,...,198,198,0,35,32,3,0,0,63.6,3.6


In [5]:
df_freshman.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과\n(모집단위)', '구분',
       '학과특성', '학과상태', '입학정원\n(A)', '모집인원 계', '모집인원 정원내\n(B)', '모집인원 정원외',
       '지원자 계', '지원자 정원내\n(C)', '지원자 정원외', '입학자 계', '입학자 정원내(D) 남',
       '입학자 정원내(D) 여', '입학자 정원외 남', '입학자 정원외 여',
       '정원내\n신입생 충원율(%)\n(D/B) × 100', '경쟁률\n(C/B)'],
      dtype='object')

In [6]:
#야간 제외
df_freshman_noon = df_freshman[(~df_freshman['구분'].str.contains('야간', na=False))]
#학석사통합과정제외
df_freshman_general = df_freshman_noon[(~df_freshman_noon['학과특성'].str.contains('학석사', na = False))]

In [7]:
def create_summarized_dataframe_for_freshman(df_gw):
    # 필요한 컬럼만 선택
    summary_df = df_gw[['기준연도','학교', '단과대학','학과\n(모집단위)','정원내\n신입생 충원율(%)\n(D/B) × 100','경쟁률\n(C/B)']].copy()
    return summary_df

df_freshman_selected = create_summarized_dataframe_for_freshman(df_freshman_general)

In [8]:
df_freshman_selected.rename(columns={'학과\n(모집단위)': '학과'}, inplace=True)
df_freshman_selected.rename(columns={'정원내\n신입생 충원율(%)\n(D/B) × 100': '신입생 충원율(%)'}, inplace=True)
df_freshman_selected.rename(columns={'경쟁률\n(C/B)': '신입생 경쟁률'}, inplace=True)

In [9]:
df_freshman_selected

,기준연도,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률
0,2023,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0
1,2023,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4
2,2023,가야대학교(김해),단과대구분없음,방사선학과,100.0,7.3
3,2023,가야대학교(김해),단과대구분없음,사회복지상담학과,50.0,4.4
4,2023,가야대학교(김해),단과대구분없음,스포츠재활복지학부,63.6,3.6
...,...,...,...,...,...,...
7532,2023,화성의과학대학교,단과대구분없음,의생명과학과,100.0,5.9
7533,2023,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,51.6,3.5
7534,2023,화신사이버대학교,단과대구분없음,글로벌교육문화학부,0.0,0.0
7535,2023,화신사이버대학교,단과대구분없음,사회서비스계열,0.0,0.0


### 중도탈락률

In [10]:
df_dropout = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\중도탈락률\2024년 _대학_4-사. 중도탈락 학생 현황_학과별자료.xlsx", header=[3,4])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [11]:
df_dropout.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(전공),구분,학과특성,...,사유별 중도탈락 학생 계(B),사유별 중도탈락 학생 미등록,사유별 중도탈락 학생 미복학,사유별 중도탈락 학생 자퇴,사유별 중도탈락 학생 학사경고,사유별 중도탈락 학생 학생활동,사유별 중도탈락 학생 유급제적,사유별 중도탈락 학생 재학연한초과,사유별 중도탈락 학생 기타,중도탈락학생비율(%)\n(B/A) × 100
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,36,2,3,31,0,0,0,0,0,4.5
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,8,1,1,6,0,0,0,0,0,16.7
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,29,2,9,18,0,0,0,0,0,31.9
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,주간,일반과정,...,2,0,1,1,0,0,0,0,0,8.7
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,16,3,1,12,0,0,0,0,0,5.6


In [12]:
#기준연도 정수로 변경
df_dropout['기준연도'] = df_dropout['기준연도'].astype(int)
#야간 제외
df_dropout_noon = df_dropout[~df_dropout['구분'].str.contains('야간', na=False)]
#학석사 제외
df_dropout_general = df_dropout_noon[~df_dropout_noon['학과특성'].str.contains('학석사', na = False)]

In [13]:
#필요한 열 선택
df_dropout_selected = df_dropout_general[['기준연도','학교','단과대학','학과\n(전공)','중도탈락학생비율(%)\n(B/A) × 100']]

In [14]:
#컬럼명 변경
df_dropout_selected.rename(columns = {'학과\n(전공)':'학과'}, inplace = True)
df_dropout_selected.rename(columns = {'중도탈락학생비율(%)\n(B/A) × 100':'중도탈락률(%)'}, inplace = True)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3828569084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropout_selected.rename(columns = {'학과\n(전공)':'학과'}, inplace = True)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3828569084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropout_selected.rename(columns = {'중도탈락학생비율(%)\n(B/A) × 100':'중도탈락률(%)'}, inplace = True)


In [15]:
df_dropout_selected

,기준연도,학교,단과대학,학과,중도탈락률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,4.5
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,16.7
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.9
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,8.7
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,5.6
...,...,...,...,...,...
13486,2023,화성의과학대학교,단과대구분없음,의생명과학과,36.4
13487,2023,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,26.7
13488,2023,화성의과학대학교,단과대구분없음,컴퓨터학과,20.0
13489,2023,화신사이버대학교,단과대구분없음,글로벌교육문화학부,14.3


### 졸업생 취업률 및 진학률

In [16]:
df_gradu = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\졸업생 취업률 진학률\2024년 _대학_5-다. 졸업생의 취업 현황_학과별자료.xlsx", header=[3,4,5,6,7])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [17]:
df_gradu.head()

,기준연도,학교종류,설립구분,지역,상태,학교명,단과대학,학과(전공),구분,학과특성,...,유지\n취업률 1차 여,유지\n취업률 2차 계,유지\n취업률 2차 남,유지\n취업률 2차 여,유지\n취업률 3차 계,유지\n취업률 3차 남,유지\n취업률 3차 여,유지\n취업률 4차 계,유지\n취업률 4차 남,유지\n취업률 4차 여
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,95.9,90.1,82.1,91.9,87.4,71.4,91.1,84.1,71.4,87.0
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,0.0,89.5,94.4,0.0,78.9,83.3,0.0,78.9,83.3,0.0
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,100.0,80.0,75.0,100.0,80.0,75.0,100.0,60.0,75.0,0.0
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,주간,일반과정,...,100.0,100.0,100.0,100.0,90.0,83.3,100.0,70.0,66.7,75.0
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,81.8,84.8,95.8,72.7,84.8,95.8,72.7,84.8,95.8,72.7


In [18]:
#기준연도 정수로 변경
df_gradu['기준연도'] = df_gradu['기준연도'].astype(int)

#야간 제외
df_gradu_noon = df_gradu[~df_gradu['구분'].str.contains('야간', na=False)]

#학석사 제외
df_gradu_general = df_gradu_noon[~df_gradu_noon['학과특성'].str.contains('학석사', na = False)]

In [19]:
df_gradu_general.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교명', '단과대학', '학과(전공)', '구분',
       '학과특성', '졸업자(A) 남', '졸업자(A) 여', '취업자(B) 건강보험\n직장가입자 남',
       '취업자(B) 건강보험\n직장가입자 여', '취업자(B) 해외\n취업자 남', '취업자(B) 해외\n취업자 여',
       '취업자(B) 농림어업\n종사자 남', '취업자(B) 농림어업\n종사자 여', '취업자(B) 개인창작\n활동\n종사자 남',
       '취업자(B) 개인창작\n활동\n종사자 여', '취업자(B) 1인\n창(사)업자 남', '취업자(B) 1인\n창(사)업자 여',
       '취업자(B) 프리랜서 남', '취업자(B) 프리랜서 여', '진학자(C) 남', '진학자(C) 여', '입대자(D)',
       '취업불가능자(E) 남', '취업불가능자(E) 여', '외국인유학생(F) 남', '외국인유학생(F) 여',
       '제외인정자(G) 남', '제외인정자(G) 여', '기타 남', '기타 여', '미상 남', '미상 여',
       '취업률(%)\n[B/{A-(C+D+E+F+G)x100', '입학\n당시\n기취업자 남', '입학\n당시\n기취업자 여',
       '교내취업자 남', '교내취업자 여', '유지\n취업률 1차 계', '유지\n취업률 1차 남', '유지\n취업률 1차 여',
       '유지\n취업률 2차 계', '유지\n취업률 2차 남', '유지\n취업률 2차 여', '유지\n취업률 3차 계',
       '유지\n취업률 3차 남', '유지\n취업률 3차 여', '유지\n취업률 4차 계', '유지\n취업률 4차 남',
       '유지\n취업률 4차 여'],
      dtype='object')

In [20]:
#필요한 열 선택
df_gradu_selected = df_gradu_general[['기준연도', '학교명', '단과대학', '학과(전공)', '졸업자(A) 남', '졸업자(A) 여', '취업률(%)\n[B/{A-(C+D+E+F+G)x100',  '진학자(C) 남', '진학자(C) 여']].copy()

In [21]:
#진학률 계산하기
df_gradu_selected['졸업자 수'] = df_gradu_selected['졸업자(A) 남'] + df_gradu_selected['졸업자(A) 여']
df_gradu_selected['진학자 수'] = df_gradu_selected['진학자(C) 남'] + df_gradu_selected['진학자(C) 여']
df_gradu_selected['졸업생 진학률(%)'] = (df_gradu_selected['진학자 수'] / df_gradu_selected['졸업자 수'] * 100).round(2)

In [22]:
df_gradu_selected

,기준연도,학교명,단과대학,학과(전공),졸업자(A) 남,졸업자(A) 여,취업률(%)\n[B/{A-(C+D+E+F+G)x100,진학자(C) 남,진학자(C) 여,졸업자 수,진학자 수,졸업생 진학률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,35,142,85.9,0,0,177,0,0.00
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,28,2,70.4,0,0,30,0,0.00
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,10,7,31.3,0,0,17,0,0.00
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,7,14,57.9,0,0,21,0,0.00
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,28,25,88.7,0,0,53,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
9910,2023,화성의과학대학교,단과대구분없음,생명과학과,3,0,0.0,0,0,3,0,0.00
9911,2023,화성의과학대학교,단과대구분없음,스포츠레저학과,7,1,100.0,1,0,8,1,12.50
9912,2023,화성의과학대학교,단과대구분없음,컴퓨터학과,9,2,50.0,0,0,11,0,0.00
9913,2023,화신사이버대학교,단과대구분없음,글로벌교육문화학부,59,90,67.4,4,2,149,6,4.03


In [23]:
#컬럼명 변경
df_gradu_selected.rename(columns = {'학교명' : '학교'}, inplace = True)
df_gradu_selected.rename(columns = {'학과(전공)' : '학과'}, inplace = True)
df_gradu_selected.rename(columns = {'취업률(%)\n[B/{A-(C+D+E+F+G)x100' : '졸업생 취업률(%)'}, inplace = True)

In [24]:
#필요한 열 재선택
df_gradu_final = df_gradu_selected[['기준연도','학교','단과대학','학과','졸업생 취업률(%)','졸업생 진학률(%)']].copy()

In [25]:
df_gradu_final

,기준연도,학교,단과대학,학과,졸업생 취업률(%),졸업생 진학률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,85.9,0.00
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,70.4,0.00
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.3,0.00
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,57.9,0.00
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,88.7,0.00
...,...,...,...,...,...,...
9910,2023,화성의과학대학교,단과대구분없음,생명과학과,0.0,0.00
9911,2023,화성의과학대학교,단과대구분없음,스포츠레저학과,100.0,12.50
9912,2023,화성의과학대학교,단과대구분없음,컴퓨터학과,50.0,0.00
9913,2023,화신사이버대학교,단과대구분없음,글로벌교육문화학부,67.4,4.03


### 캡스톤디자인

In [26]:
#데이터 불러오기
caps = pd.read_excel(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\캡스톤디자인\20250915_12-카-2 캡스톤 디자인 운영 현황_대학(2023~2025).xlsx", sheet_name ='2024')

In [27]:
caps.head()

,기준년도,학교종류명,학교설립구분명,공시지역명,학교상태명,학교명,본분교구분명,학과명,계열대분류명,계열중분류명,...,학점수,이수학생수_해당학과,이수학생수_타학과 계열_인문사회,이수학생수_타학과 계열_자연과학,이수학생수_타학과 계열_공학,이수학생수_타학과 계열_의학,이수학생수_타학과 계열_예체능,지원금수령학생수,지원금수령총금액_시제(작)품 제작비,지원금수령총금액_교육프로그램 지원비
0,2023,대학교,국립,강원,기존,국립강릉원주대학교,본교,회계학과,인문사회계열,경영・경제,...,3,39,0.0,0,0,0,0,39,0,2170000
1,2023,대학교,국립,강원,기존,국립강릉원주대학교,본교,회계학과,인문사회계열,경영・경제,...,3,25,0.0,0,0,0,0,25,0,1939400
2,2023,대학교,국립,강원,기존,국립강릉원주대학교,본교,경제학과,인문사회계열,경영・경제,...,3,17,0.0,0,0,0,0,16,0,1218620
3,2023,대학교,국립,강원,기존,국립강릉원주대학교,본교,관광경영학과,인문사회계열,경영・경제,...,3,37,2.0,0,0,0,0,37,2605600,3525200
4,2023,대학교,국립,강원,기존,국립강릉원주대학교,본교,경영학과,인문사회계열,경영・경제,...,3,31,0.0,2,0,0,0,25,200000,1800000


In [28]:
#계열명 확인
caps['계열대분류명'].unique()

array(['인문사회계열', '자연과학계열', '공학계열', '예체능계열', '의학계열'], dtype=object)

In [29]:
caps.columns
#'학교명', '본분교구분명', '학과명','계열대분류명','단과대학명','주야간구분명','학기구분명','이수학생수_해당학과'

Index(['기준년도', '학교종류명', '학교설립구분명', '공시지역명', '학교상태명', '학교명', '본분교구분명', '학과명',
       '계열대분류명', '계열중분류명', '계열소분류명', '자체계열명', '단과대학명', '주야간구분명', '학기구분명',
       '학점수', '이수학생수_해당학과', '이수학생수_타학과 계열_인문사회', '이수학생수_타학과 계열_자연과학',
       '이수학생수_타학과 계열_공학', '이수학생수_타학과 계열_의학', '이수학생수_타학과 계열_예체능', '지원금수령학생수',
       '지원금수령총금액_시제(작)품 제작비', '지원금수령총금액_교육프로그램 지원비'],
      dtype='object')

In [30]:
# 필요한 열 선택
def create_summarized_dataframe(df):
    # 필요한 컬럼만 선택
    summary_df = df[['학교명', '본분교구분명', '학과명','계열대분류명','단과대학명','주야간구분명','학기구분명','이수학생수_해당학과']].copy()
    return summary_df

summary_caps = create_summarized_dataframe(caps)


In [31]:
#야간제외
weekly_caps = summary_caps[summary_caps['주야간구분명'] == '주간']

In [32]:
weekly_caps

,학교명,본분교구분명,학과명,계열대분류명,단과대학명,주야간구분명,학기구분명,이수학생수_해당학과
0,국립강릉원주대학교,본교,회계학과,인문사회계열,사회과학대학,주간,2학기,39
1,국립강릉원주대학교,본교,회계학과,인문사회계열,사회과학대학,주간,1학기,25
2,국립강릉원주대학교,본교,경제학과,인문사회계열,사회과학대학,주간,1학기,17
3,국립강릉원주대학교,본교,관광경영학과,인문사회계열,사회과학대학,주간,1학기,37
4,국립강릉원주대학교,본교,경영학과,인문사회계열,사회과학대학,주간,2학기,31
...,...,...,...,...,...,...,...,...
6838,상명대학교,제2캠퍼스,텍스타일디자인전공,예체능계열,디자인대학,주간,1학기,11
6839,상명대학교,제2캠퍼스,정보보안공학과,공학계열,공과대학,주간,1학기,35
6840,상명대학교,제2캠퍼스,소프트웨어학과,공학계열,공과대학,주간,1학기,48
6841,상명대학교,제2캠퍼스,스페이스디자인전공,예체능계열,디자인대학,주간,2학기,32


In [33]:
# 그룹화하여 평균 계산
caps_mean = (
    weekly_caps.groupby(["학교명", "단과대학명", "학과명"], as_index=False)
      .agg({
          "본분교구분명" : 'first',  # 그룹 내 첫 번째 값 유지
          "이수학생수_해당학과": "mean"
      })
)


In [34]:
#컬럼명 변경
caps_mean.rename(columns = {'학교명' : '학교',
                            '단과대학명' : '단과대학',
                            '학과명' : '학과',
                            '계열대분류명' : '계열'}, inplace = True)

In [35]:
#학교명에 본분교구분넣기
caps_mean["학교"] = caps_mean.apply(
    lambda x: x["학교"] if x["본분교구분명"] == "본교" 
    else f"{x['학교']}_{x['본분교구분명']}",
    axis=1
)

In [36]:
caps_mean

,학교,단과대학,학과,본분교구분명,이수학생수_해당학과
0,가야대학교(김해),단과대구분없음,외식조리영양학부,본교,8.000000
1,가천대학교,IT융합대학,의공학과,본교,1.000000
2,가천대학교,IT융합대학,전기공학과,본교,74.500000
3,가천대학교,IT융합대학,전자공학과,본교,87.000000
4,가천대학교,IT융합대학,컴퓨터공학과,본교,19.000000
...,...,...,...,...,...
3021,홍익대학교,미술대학,도예ㆍ유리과,본교,12.000000
3022,홍익대학교,미술대학,동양화과,본교,7.000000
3023,홍익대학교,미술대학,디자인학부 산업디자인전공,본교,57.000000
3024,홍익대학교,미술대학,섬유미술ㆍ패션디자인과,본교,24.333333


### 연구실적

In [37]:
#데이터 불러오기
df_research = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\교원 1인당 연구실적\전임교원의 연구 실적 (대학)_2025-07-03133559666.xlsx", header=[3,4,5,6,7])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [38]:
df_research.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과,전임\n교원 계,전임\n교원 남,...,전임교원 1인당 논문 실적 국제\n기준 여,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 남,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 여,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 남,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 여,전임교원 1인당 저∙역서 실적 계,전임교원 1인당 저∙역서 실적 남,전임교원 1인당 저∙역서 실적 여
0,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,6,5,...,0.0000,0.7500,0.5200,1.9000,0.0000,0.0,0.0000,0.1833,0.22,0.0000
1,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,3,3,...,0.0000,0.6667,0.6667,0.0000,0.0000,0.0,0.0000,0.0000,0.00,0.0000
2,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,교양학부,6,4,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.00,0.0000
3,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,20,1,...,0.0624,0.5446,0.0000,0.5733,0.0393,0.0,0.0414,0.0867,0.00,0.0912
4,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,안경광학과,1,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,1.0000,1.00,0.0000


In [39]:
def create_summarized_dataframe(df_gw):
    # 필요한 컬럼만 선택
    summary_df = df_gw[['기준연도','학교', '단과대학', '학과','전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계',  '전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계']].copy()
    return summary_df

df_research_summary = create_summarized_dataframe(df_research)

In [40]:
df_research_summary.head()

,기준연도,학교,단과대학,학과,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계
0,2023,가야대학교(김해),단과대구분없음,방사선학과,0.7500,0.0000
1,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,0.6667,0.0000
2,2023,가야대학교(김해),단과대구분없음,교양학부,0.0000,0.0000
3,2023,가야대학교(김해),단과대구분없음,간호학과,0.5446,0.0393
4,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0000,0.0000


### 연구비 수혜 실적

In [41]:
#데이터 불러오기
df_research_money = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\교원1인당 연구비\2024년 _대학_12-가. 연구비 수혜 실적_학과별자료.xlsx", header=[3,4,5,6,7])

#기준연도 정수로 변경
df_research_money['기준연도'] = df_research_money['기준연도'].astype(int)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [42]:
#필요한 열 선택
df_research_money_summary = df_research_money[['기준연도','학교', '단과대학', '학과','전임\n교원\n수 남','전임\n교원\n수 여','연구비 지원 계 연구비 여','연구비 지원 계 연구비 남', '대응자금 교내 남', '대응자금 교내 여', '대응자금 교외 남', '대응자금 교외 여']].copy()

In [43]:
df_research_money_summary.head()

,기준연도,학교,단과대학,학과,전임\n교원\n수 남,전임\n교원\n수 여,연구비 지원 계 연구비 여,연구비 지원 계 연구비 남,대응자금 교내 남,대응자금 교내 여,대응자금 교외 남,대응자금 교외 여
0,2023,가야대학교(김해),단과대구분없음,방사선학과,5,1,2500,5000,0,0,0,0
1,2023,가야대학교(김해),단과대구분없음,안경광학과,1,0,0,0,0,0,0,0
2,2023,가야대학교(김해),단과대구분없음,간호학과,1,19,15500,2500,0,0,0,0
3,2023,가야대학교(김해),단과대구분없음,교양학부,4,2,0,0,0,0,0,0
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,3,0,0,4500,0,0,0,0


In [44]:
#총연구비계산
df_research_money_summary['총연구비'] = df_research_money_summary['연구비 지원 계 연구비 여'] + df_research_money_summary['연구비 지원 계 연구비 남'] + \
    df_research_money_summary['대응자금 교내 여'] + df_research_money_summary['대응자금 교내 남'] + \
    df_research_money_summary['대응자금 교외 여'] + df_research_money_summary['대응자금 교외 남']

#총교원수 계산
df_research_money_summary['총교원수'] = df_research_money_summary['전임\n교원\n수 남'] + df_research_money_summary['전임\n교원\n수 여']


#1인당 연구비 계산
df_research_money_summary['1인당 연구비(천원)'] = (df_research_money_summary['총연구비'] / df_research_money_summary['총교원수']).round(2)

In [45]:
df_research_money_summary.head()

,기준연도,학교,단과대학,학과,전임\n교원\n수 남,전임\n교원\n수 여,연구비 지원 계 연구비 여,연구비 지원 계 연구비 남,대응자금 교내 남,대응자금 교내 여,대응자금 교외 남,대응자금 교외 여,총연구비,총교원수,1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,5,1,2500,5000,0,0,0,0,7500,6,1250.0
1,2023,가야대학교(김해),단과대구분없음,안경광학과,1,0,0,0,0,0,0,0,0,1,0.0
2,2023,가야대학교(김해),단과대구분없음,간호학과,1,19,15500,2500,0,0,0,0,18000,20,900.0
3,2023,가야대학교(김해),단과대구분없음,교양학부,4,2,0,0,0,0,0,0,0,6,0.0
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,3,0,0,4500,0,0,0,0,4500,3,1500.0


In [46]:
df_research_money_final = df_research_money_summary[['기준연도', '학교', '단과대학', '학과', '1인당 연구비(천원)']].copy()

In [47]:
df_research_money_final.head()

,기준연도,학교,단과대학,학과,1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,1250.0
1,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0
2,2023,가야대학교(김해),단과대구분없음,간호학과,900.0
3,2023,가야대학교(김해),단과대구분없음,교양학부,0.0
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,1500.0


### 재학생 충원율

In [48]:
#재학생충원율 데이터 불러오기
df_student_2022_sec_and_2023_first = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\재학생충원율\2023년 _대학_4-라-1. 재학생 충원율_학과별자료.xlsx", header=[3,4])
df_student_2023_sec_and_2024_first = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\재학생충원율\2024년 _대학_4-라-1. 재학생 충원율_학과별자료.xlsx", header=[3,4])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [49]:
#기준연도 2023 상반기, 하반기만 선택
df_student_2023_first = df_student_2022_sec_and_2023_first[df_student_2022_sec_and_2023_first['기준연도'] == '2023 년 상반기']
df_student_2023_sec = df_student_2023_sec_and_2024_first[df_student_2023_sec_and_2024_first['기준연도'] == '2023 년 하반기']

In [50]:
def create_summarized_dataframe(df):
    # 필요한 컬럼만 선택
    summary_df = df[['기준연도','학교', '계열','재학생충원율(%){C/(A-B)}X100','정원내 재학생 충원율(%){D/(A-B)}X100']].copy()
    return summary_df

df_student_2023_first_filtered = create_summarized_dataframe(df_student_2023_first)
df_student_2023_sec_filtered = create_summarized_dataframe(df_student_2023_sec)

In [51]:
# 두 DataFrame을 수직 결합
df_all = pd.concat([df_student_2023_first_filtered, df_student_2023_sec_filtered], ignore_index=True)

# 기준연도에서 연도(숫자)만 추출
df_all['연도'] = df_all['기준연도'].str.extract(r'(\d{4})')

In [52]:
# 연도 및 학교 기준으로 평균 계산
df_avg = df_all.groupby(['연도', '학교','계열'])[
    ['재학생충원율(%){C/(A-B)}X100', '정원내 재학생 충원율(%){D/(A-B)}X100']
].mean().reset_index()

In [53]:
df_avg

,연도,학교,계열,재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100
0,2023,가야대학교(고령) _제2캠퍼스,공학계열,0.0,0.0
1,2023,가야대학교(고령) _제2캠퍼스,예ㆍ체능계열,0.0,0.0
2,2023,가야대학교(고령) _제2캠퍼스,의학계열,0.0,0.0
3,2023,가야대학교(고령) _제2캠퍼스,인문ㆍ사회계열,0.0,0.0
4,2023,가야대학교(고령) _제2캠퍼스,자연과학계열,0.0,0.0
...,...,...,...,...,...
1250,2023,화신사이버대학교,공학계열,0.0,0.0
1251,2023,화신사이버대학교,예ㆍ체능계열,0.0,0.0
1252,2023,화신사이버대학교,의학계열,0.0,0.0
1253,2023,화신사이버대학교,인문ㆍ사회계열,113.5,87.8


In [54]:
df_avg_cleaned = clean_text_columns(df_avg,['학교','계열'])

In [55]:
#계열 데이터 불러오기
df_line = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\지표new-6페이지\2023년 _대학_4-마. 재적 학생 현황_학과별자료.xlsx", header = [3,4,5])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [1229]:
df_line.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(전공),구분,학과특성,...,학사학위취득유예학생(C) 남 정원내,학사학위취득유예학생(C) 남 정원외,학사학위취득유예학생(C) 여 정원내,학사학위취득유예학생(C) 여 정원외,재적학생(D=A+B+C) 계 정원내,재적학생(D=A+B+C) 계 정원외,재적학생(D=A+B+C) 남 정원내,재적학생(D=A+B+C) 남 정원외,재적학생(D=A+B+C) 여 정원내,재적학생(D=A+B+C) 여 정원외
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,주간,일반과정,...,0,0,0,0,225,8,157,6,68,2
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,0,0,0,0,666,133,226,35,440,98
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,0,0,0,0,45,3,42,1,3,2
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,0,0,0,0,91,0,82,0,9,0
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰행정학과,주간,일반과정,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df_line.columns #여기부터 다시하기

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과\n(전공)', '구분',
       '학과특성', '학과상태', '계열', '학생정원', '재학생(A) 계 정원내', '재학생(A) 계 정원외',
       '재학생(A) 남 정원내', '재학생(A) 남 정원외', '재학생(A) 여 정원내', '재학생(A) 여 정원외',
       '휴학생(B) 계 정원내', '휴학생(B) 계 정원외', '휴학생(B) 남 정원내', '휴학생(B) 남 정원외',
       '휴학생(B) 여 정원내', '휴학생(B) 여 정원외', '학사학위취득유예학생(C) 계 정원내',
       '학사학위취득유예학생(C) 계 정원외', '학사학위취득유예학생(C) 남 정원내', '학사학위취득유예학생(C) 남 정원외',
       '학사학위취득유예학생(C) 여 정원내', '학사학위취득유예학생(C) 여 정원외', '재적학생(D=A+B+C) 계 정원내',
       '재적학생(D=A+B+C) 계 정원외', '재적학생(D=A+B+C) 남 정원내', '재적학생(D=A+B+C) 남 정원외',
       '재적학생(D=A+B+C) 여 정원내', '재적학생(D=A+B+C) 여 정원외'],
      dtype='object')

In [62]:
df_line = df_line[df_line['학과특성'] != '학석사통합과정']

In [66]:
df_line_selected = df_line[['학교', '단과대학', '학과\n(전공)', '구분','계열', '학생정원','학과상태']].copy()
df_line_selected = df_line_selected[df_line_selected['구분'] != '야간']
df_line_selected.rename(columns = {'학과\n(전공)' : '학과',
                                   '학과상태' : '학과상태_2023'}, inplace = True)

In [67]:
#clean text
df_line_selected_clean = clean_text_columns(df_line_selected, ['학교','단과대학','학과','계열'])

In [68]:
df_line_selected_clean

,학교,단과대학,학과,구분,계열,학생정원,학과상태_2023
0,가야대학교(김해),단과대구분없음,방사선학과,주간,자연과학계열,180,기존
1,가야대학교(김해),단과대구분없음,간호학과,주간,자연과학계열,602,기존
2,가야대학교(김해),단과대구분없음,경영물류학과,주간,인문사회계열,65,폐과
3,가야대학교(김해),단과대구분없음,경찰소방학과,주간,인문사회계열,30,폐과
4,가야대학교(김해),단과대구분없음,경찰행정학과,주간,인문사회계열,80,폐과
...,...,...,...,...,...,...,...
14429,화성의과학대학교,단과대구분없음,컴퓨터학과,주간,자연과학계열,92,폐과
14430,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,주간,자연과학계열,30,변경
14431,화신사이버대학교,단과대구분없음,상담복지학부,원격,인문사회계열,0,기존
14432,화신사이버대학교,단과대구분없음,글로벌교육문화학부,원격,인문사회계열,0,기존


### 캡스톤디자인, 재학생충원율 제외하고 병합하기

In [70]:
#신입생충원율 및 경쟁률 데이터 확인하기
df_freshman_selected.head()

,기준연도,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률
0,2023,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0
1,2023,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4
2,2023,가야대학교(김해),단과대구분없음,방사선학과,100.0,7.3
3,2023,가야대학교(김해),단과대구분없음,사회복지상담학과,50.0,4.4
4,2023,가야대학교(김해),단과대구분없음,스포츠재활복지학부,63.6,3.6


In [71]:
#중도탈락률 데이터 확인하기
df_dropout_selected.head()

,기준연도,학교,단과대학,학과,중도탈락률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,4.5
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,16.7
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.9
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,8.7
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,5.6


In [72]:
# 전처리 적용
df_freshman_selected_and_clean = clean_text_columns(df_freshman_selected, ['학교', '학과', '기준연도','단과대학'])
df_dropout_selected_and_clean = clean_text_columns(df_dropout_selected, ['학교', '학과', '기준연도','단과대학'])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\1875311623.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\1875311623.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('·', '', regex=False)   # U+00B7
C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\1875311623.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [73]:
# 병합
df_merged_step_one = pd.merge(
    df_freshman_selected_and_clean,
    df_dropout_selected_and_clean,
    on=['학교', '학과', '기준연도','단과대학'],
    how='outer'
)

In [74]:
df_merged_step_one

,기준연도,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4,5.6
...,...,...,...,...,...,...,...
13295,2023,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,51.6,3.5,26.7
13296,2023,화성의과학대학교,단과대구분없음,컴퓨터학과,NaN,NaN,20.0
13297,2023,화신사이버대학교,단과대구분없음,글로벌교육문화학부,0.0,0.0,14.3
13298,2023,화신사이버대학교,단과대구분없음,사회서비스계열,0.0,0.0,NaN


In [75]:
df_merged_step_one['학교'].unique()

array(['가야대학교(김해)', '가천대학교', '가톨릭관동대학교', '가톨릭꽃동네대학교', '가톨릭대학교',
       '가톨릭대학교_제2캠퍼스', '가톨릭대학교_제3캠퍼스', '감리교신학대학교', '강남대학교', '강서대학교',
       '강원대학교', '강원대학교_제2캠퍼스', '건국대학교', '건국대학교(글로컬)_분교', '건양대학교',
       '건양대학교_제2캠퍼스', '건양사이버대학교', '경기대학교', '경기대학교_제2캠퍼스', '경남과학기술대학교',
       '경남대학교', '경동대학교', '경동대학교_제3캠퍼스', '경동대학교_제4캠퍼스', '경북대학교', '경상국립대학교',
       '경성대학교', '경운대학교', '경인교육대학교', '경인교육대학교_제2캠퍼스', '경일대학교', '경희대학교',
       '경희사이버대학교', '계명대학교', '고려대학교', '고려대학교(세종)_분교', '고려사이버대학교', '고신대학교',
       '공주교육대학교', '광신대학교', '광운대학교', '광주가톨릭대학교', '광주과학기술원', '광주교육대학교',
       '광주대학교', '광주여자대학교', '국립강릉원주대학교', '국립강릉원주대학교_제2캠퍼스', '국립경국대학교',
       '국립공주대학교', '국립군산대학교', '국립금오공과대학교', '국립목포대학교', '국립목포해양대학교',
       '국립부경대학교', '국립순천대학교', '국립창원대학교', '국립한국교통대학교', '국립한국해양대학교',
       '국립한밭대학교', '국민대학교', '국제사이버대학교', '극동대학교', '글로벌사이버대학교', '금강대학교',
       '김천대학교', '나사렛대학교', '남부대학교', '남서울대학교', '단국대학교', '단국대학교_제2캠퍼스',
       '대구가톨릭대학교', '대구경북과학기술원', '대구교육대학교', '대구대학교', '대구사이버대학교', '대구예술대학교',
       '대구한의대학교', '대신대학교

In [76]:
#졸업생 취업률 및 진학률 데이터 확인하기
df_gradu_final.head()

,기준연도,학교,단과대학,학과,졸업생 취업률(%),졸업생 진학률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,85.9,0.0
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,70.4,0.0
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.3,0.0
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,57.9,0.0
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,88.7,0.0


In [77]:
#데이터 공백 및 문자열 정리하기
df_gradu_final_and_clean = clean_text_columns(df_gradu_final, ['학교','학과','단과대학'])

In [78]:
# 병합
df_merged_freshman_dropout_gradu = pd.merge(
    df_merged_step_one,
    df_gradu_final_and_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [79]:
df_merged_freshman_dropout_gradu.drop(columns=['기준연도_x'], inplace=True)
df_merged_freshman_dropout_gradu.drop(columns=['기준연도_y'], inplace=True)

In [80]:
df_merged_freshman_dropout_gradu

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%)
0,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5,85.9,0.00
1,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7,70.4,0.00
2,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9,31.3,0.00
3,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7,57.9,0.00
4,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4,5.6,88.7,0.00
...,...,...,...,...,...,...,...,...
14297,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,51.6,3.5,26.7,NaN,NaN
14298,화성의과학대학교,단과대구분없음,컴퓨터학과,NaN,NaN,20.0,50.0,0.00
14299,화신사이버대학교,단과대구분없음,글로벌교육문화학부,0.0,0.0,14.3,67.4,4.03
14300,화신사이버대학교,단과대구분없음,사회서비스계열,0.0,0.0,NaN,NaN,NaN


In [81]:
# 연구 실적 데이터 확인하기
df_research_summary.head()

,기준연도,학교,단과대학,학과,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계
0,2023,가야대학교(김해),단과대구분없음,방사선학과,0.7500,0.0000
1,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,0.6667,0.0000
2,2023,가야대학교(김해),단과대구분없음,교양학부,0.0000,0.0000
3,2023,가야대학교(김해),단과대구분없음,간호학과,0.5446,0.0393
4,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0000,0.0000


In [82]:
df_research_summary_clean = clean_text_columns(df_research_summary, ['학교', '학과','단과대학'])

In [83]:
df_step_three_merged = pd.merge(
    df_merged_freshman_dropout_gradu,
    df_research_summary_clean,
    on=['학교', '학과','단과대학'],
    how='outer')

In [84]:
df_step_three_merged.drop(columns=['기준연도'], inplace=True)

In [85]:
df_step_three_merged
#신입생 충원율, 경쟁률, 중도탈락률, 졸업생 취업률 및 진학률, 연구 실적 데이터 병합 완료

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계
0,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5,85.9,0.0,0.5446,0.0393
1,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000
2,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9,31.3,0.0,0.3333,0.0000
3,가야대학교(김해),단과대구분없음,교양학부,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
4,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7,57.9,0.0,0.6667,0.0000
...,...,...,...,...,...,...,...,...,...,...
16794,화신사이버대학교,단과대구분없음,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
16795,화신사이버대학교,단과대구분없음,실용외국어학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
16796,화신사이버대학교,단과대구분없음,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
16797,화신사이버대학교,단과대구분없음,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000


In [86]:
# 연구비 수혜 실적 데이터 확인하기
df_research_money_final.head()

,기준연도,학교,단과대학,학과,1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,1250.0
1,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0
2,2023,가야대학교(김해),단과대구분없음,간호학과,900.0
3,2023,가야대학교(김해),단과대구분없음,교양학부,0.0
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,1500.0


In [87]:
df_research_money_final.drop(columns=['기준연도'], inplace=True)

In [88]:
df_research_money_final['학교'].unique()

array(['가야대학교(김해) ', '가천대학교 ', '가천대학교 교육대학원 ', '가천대학교 사회정책대학원 ',
       '가천대학교 일반대학원 ', '가천대학교 특수치료대학원 ', '가톨릭관동대학교 ', '가톨릭꽃동네대학교 ',
       '가톨릭대학교 ', '가톨릭대학교 _제2캠퍼스', '가톨릭대학교 _제3캠퍼스', '가톨릭대학교 교육대학원 ',
       '가톨릭대학교 교회법대학원 ', '가톨릭대학교 교회음악대학원 _제3캠퍼스', '가톨릭대학교 대학원 ',
       '가톨릭대학교 문화영성대학원 ', '가톨릭대학교 보건의료경영대학원 _제2캠퍼스', '가톨릭대학교 사회복지대학원 ',
       '가톨릭대학교 상담심리대학원 ', '가톨릭대학교 생명대학원 _제2캠퍼스', '감리교신학대학교 ', '강남대학교 ',
       '강남대학교 교육대학원 ', '강남대학교 국제대학원 ', '강남대학교 융복합대학원 ', '강서대학교 ',
       '강원대학교 ', '강원대학교 _제2캠퍼스', '강원대학교 교육대학원 ', '강원대학교 방재전문대학원 _제2캠퍼스',
       '강원대학교 법학전문대학원 ', '강원대학교 일반대학원 ', '건국대학교 ', '건국대학교 건축전문대학원 ',
       '건국대학교 교육대학원 ', '건국대학교 법학전문대학원 ', '건국대학교 산업대학원 ',
       '건국대학교 예술디자인대학원 ', '건국대학교 일반대학원 ', '건국대학교 행정대학원 ',
       '건국대학교(글로컬) _분교', '건국대학교(글로컬) 교육대학원 _분교', '건국대학교(글로컬) 의학전문대학원 _분교',
       '건양대학교 ', '건양대학교 _제2캠퍼스', '건양사이버대학교 ', '경기대학교 ', '경기대학교 _제2캠퍼스',
       '경기대학교 공학대학원 ', '경기대학교 관광전문대학원 ', '경기대학교 교육대학원 ', '경기대학교 대체의학대학원 ',
       '경기대학교 서비스경영전문대학원 ', '경기대학교 일반대학원 ', 

In [89]:
df_research_money_final_clean = clean_text_columns(df_research_money_final, ['학교', '학과','단과대학'])

In [90]:
df_research_money_final['학교'] = df_research_money_final['학교'].str.replace(r'\s*_(제\d+캠퍼스|분교)', r'_\1', regex=True)

In [91]:
df_research_money_final_clean.head()

,학교,단과대학,학과,1인당 연구비(천원)
0,가야대학교(김해),단과대구분없음,방사선학과,1250.0
1,가야대학교(김해),단과대구분없음,안경광학과,0.0
2,가야대학교(김해),단과대구분없음,간호학과,900.0
3,가야대학교(김해),단과대구분없음,교양학부,0.0
4,가야대학교(김해),단과대구분없음,귀금속주얼리학과,1500.0


In [92]:
df_step_four_merged = pd.merge(
    df_step_three_merged,
    df_research_money_final_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [93]:
df_step_four_merged

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원)
0,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5,85.9,0.0,0.5446,0.0393,900.0
1,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000,1125.0
2,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9,31.3,0.0,0.3333,0.0000,13110.0
3,가야대학교(김해),단과대구분없음,교양학부,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0
4,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7,57.9,0.0,0.6667,0.0000,1500.0
...,...,...,...,...,...,...,...,...,...,...,...
16909,화신사이버대학교,단과대구분없음,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0
16910,화신사이버대학교,단과대구분없음,실용외국어학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0
16911,화신사이버대학교,단과대구분없음,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0
16912,화신사이버대학교,단과대구분없음,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0


In [94]:
caps_mean.head()

,학교,단과대학,학과,본분교구분명,이수학생수_해당학과
0,가야대학교(김해),단과대구분없음,외식조리영양학부,본교,8.0
1,가천대학교,IT융합대학,의공학과,본교,1.0
2,가천대학교,IT융합대학,전기공학과,본교,74.5
3,가천대학교,IT융합대학,전자공학과,본교,87.0
4,가천대학교,IT융합대학,컴퓨터공학과,본교,19.0


In [95]:
#캡스톤디자인
caps_clean = clean_text_columns(caps_mean, ['학교','단과대학','학과'])

In [ ]:
#컬럼정리
caps_clean.drop(columns = ['본분교구분명'], inplace = True)
caps_clean.rename(columns = {'이수학생수_해당학과' : '캡스톤디자인 평균이수학생수'}, inplace = True )

In [97]:
#병합하기
df_step_five_merged = pd.merge(
    df_step_four_merged,
    caps_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [98]:
df_line_selected_clean.drop(columns = ['구분'], inplace = True)

In [99]:
df_step_six_merged = pd.merge(
    df_step_five_merged,
    df_line_selected_clean,
    on=['학교', '학과','단과대학'],
    how='left'
)

In [100]:
df_step_six_merged

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원),캡스톤디자인 평균이수학생수,계열,학생정원,학과상태_2023
0,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5,85.9,0.0,0.5446,0.0393,900.0,NaN,자연과학계열,602.0,기존
1,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000,1125.0,NaN,인문사회계열,65.0,폐과
2,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9,31.3,0.0,0.3333,0.0000,13110.0,NaN,인문사회계열,30.0,폐과
3,가야대학교(김해),단과대구분없음,교양학부,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
4,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7,57.9,0.0,0.6667,0.0000,1500.0,NaN,자연과학계열,50.0,폐과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17048,화신사이버대학교,단과대구분없음,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17049,화신사이버대학교,단과대구분없음,실용외국어학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17050,화신사이버대학교,단과대구분없음,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17051,화신사이버대학교,단과대구분없음,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN


In [101]:
df_avg_cleaned.drop(columns = ['재학생충원율(%){C/(A-B)}X100'], inplace = True)
df_avg_cleaned.rename(columns = {'정원내 재학생 충원율(%){D/(A-B)}X100' : '재학생충원율'}, inplace = True)

In [102]:
df_avg_cleaned

,연도,학교,계열,재학생충원율
0,2023,가야대학교(고령)_제2캠퍼스,공학계열,0.0
1,2023,가야대학교(고령)_제2캠퍼스,예체능계열,0.0
2,2023,가야대학교(고령)_제2캠퍼스,의학계열,0.0
3,2023,가야대학교(고령)_제2캠퍼스,인문사회계열,0.0
4,2023,가야대학교(고령)_제2캠퍼스,자연과학계열,0.0
...,...,...,...,...
1250,2023,화신사이버대학교,공학계열,0.0
1251,2023,화신사이버대학교,예체능계열,0.0
1252,2023,화신사이버대학교,의학계열,0.0
1253,2023,화신사이버대학교,인문사회계열,87.8


In [103]:
df_step_final_merged = pd.merge(
    df_step_six_merged,
    df_avg_cleaned,
    on=['학교', '계열'],
    how='outer'
)

In [104]:
df_step_six_merged

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원),캡스톤디자인 평균이수학생수,계열,학생정원,학과상태_2023
0,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0,4.5,85.9,0.0,0.5446,0.0393,900.0,NaN,자연과학계열,602.0,기존
1,가야대학교(김해),단과대구분없음,경영물류학과,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000,1125.0,NaN,인문사회계열,65.0,폐과
2,가야대학교(김해),단과대구분없음,경찰소방학과,NaN,NaN,31.9,31.3,0.0,0.3333,0.0000,13110.0,NaN,인문사회계열,30.0,폐과
3,가야대학교(김해),단과대구분없음,교양학부,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
4,가야대학교(김해),단과대구분없음,귀금속주얼리학과,NaN,NaN,8.7,57.9,0.0,0.6667,0.0000,1500.0,NaN,자연과학계열,50.0,폐과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17048,화신사이버대학교,단과대구분없음,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17049,화신사이버대학교,단과대구분없음,실용외국어학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17050,화신사이버대학교,단과대구분없음,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN
17051,화신사이버대학교,단과대구분없음,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0,NaN,NaN,NaN,NaN


In [105]:
df_step_final_merged

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원),캡스톤디자인 평균이수학생수,계열,학생정원,학과상태_2023,연도,재학생충원율
0,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공학계열,NaN,NaN,2023,0.0
1,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,예체능계열,NaN,NaN,2023,0.0
2,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,의학계열,NaN,NaN,2023,0.0
3,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인문사회계열,NaN,NaN,2023,0.0
4,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자연과학계열,NaN,NaN,2023,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17471,화신사이버대학교,단과대구분없음,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
17472,화신사이버대학교,단과대구분없음,실용외국어학과,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
17473,화신사이버대학교,단과대구분없음,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
17474,화신사이버대학교,단과대구분없음,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [1129]:
# 검색은 학교 & 학과가 아닌, 학교를 검색 후 해당 학과를 찾아보는 것을 추천
# 이유는
#1. 검색 시 정확한 검색어를 입력해야함 -> 학교명은 쉬우나 학과명은 복잡함(경영학과, 경영학전공 등)
#2. 학교명은 동일하나 학과명이 다른 경우가 있음(예: 컴퓨터공학과, 컴퓨터공학전공 등)
#3. 유사학과 탐색을 위해 학교 검색을 추천

In [106]:
#대학원,사이버대 제외하기
df_merged_fianl_only_college = df_step_final_merged[~df_step_final_merged['학교'].str.contains('대학원|사이버', na=False)]

In [108]:
df_merged_fianl_only_college.head()

,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원),캡스톤디자인 평균이수학생수,계열,학생정원,학과상태_2023,연도,재학생충원율
0,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공학계열,NaN,NaN,2023,0.0
1,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,예체능계열,NaN,NaN,2023,0.0
2,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,의학계열,NaN,NaN,2023,0.0
3,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인문사회계열,NaN,NaN,2023,0.0
4,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자연과학계열,NaN,NaN,2023,0.0


In [ ]:
#학과상태 2024년도 데이터 불러오기
df_condition_real_2024 = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\통합시도\2024년 _대학_4-마. 재적 학생 현황_학과별자료.xlsx", header = [3,4,5])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_21684\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [113]:
df_condition_real_2024['학과특성'].unique()

array(['일반과정', '계약학과', '학석사통합과정', '특별과정'], dtype=object)

In [114]:
#야간제외
df_condition_real_2024 = df_condition_real_2024[df_condition_real_2024['구분']!= '야간']

#학석사 제외
df_condition_real_2024 = df_condition_real_2024[df_condition_real_2024['학과특성'] != '학석사통합과정']

In [117]:
#컬럼 정리
df_condition_real_2024_selected = df_condition_real_2024[['학교', '단과대학', '학과\n(전공)', '학과상태']].copy()
df_condition_real_2024_selected.rename(columns = {'학과\n(전공)':'학과',
                                             '학과상태' : '학과상태_2024'}, inplace = True)

df_condition_real_2024_selected_and_clean = clean_text_columns(df_condition_real_2024_selected, ['학교','학과','단과대학'])

In [118]:
df_condition_real_2024_selected

,학교,단과대학,학과,학과상태_2024
0,가야대학교(김해),단과대구분없음,사회복지상담학과(3년과정),변경
1,가야대학교(김해),단과대구분없음,작업치료학과,기존
2,가야대학교(김해),단과대구분없음,유아교육과,폐과
3,가야대학교(김해),단과대구분없음,외식조리영양학부,폐과
4,가야대학교(김해),단과대구분없음,안경광학과,폐과
...,...,...,...,...
14896,화성의과학대학교,단과대구분없음,컴퓨터학과,폐과
14897,화성의과학대학교,단과대구분없음,데이터사이언스학과,변경
14898,화신사이버대학교,단과대구분없음,상담복지학부,기존
14899,화신사이버대학교,단과대구분없음,사회서비스계열,기존


In [128]:
df_final = pd.merge(
    df_merged_fianl_only_college,
    df_condition_real_2024_selected_and_clean,
    on = ['학교','단과대학','학과'],
    how = 'left'
)

In [129]:
cols = ['학교', '단과대학', '학과', '학과상태_2023','학과상태_2024','신입생 충원율(%)', '신입생 경쟁률','재학생충원율','중도탈락률(%)','캡스톤디자인 평균이수학생수','졸업생 취업률(%)', '졸업생 진학률(%)',
        '전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계','전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계','1인당 연구비(천원)','계열', '학생정원', '연도' ]
df_final_clean = df_final[cols]

In [ ]:
#중복행제외
df_final_clean_ver = df_final_clean.drop_duplicates()

In [131]:
df_final_clean_ver

,학교,단과대학,학과,학과상태_2023,학과상태_2024,신입생 충원율(%),신입생 경쟁률,재학생충원율,중도탈락률(%),캡스톤디자인 평균이수학생수,졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,1인당 연구비(천원),계열,학생정원,연도
0,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공학계열,NaN,2023
1,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,예체능계열,NaN,2023
2,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,의학계열,NaN,2023
3,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인문사회계열,NaN,2023
4,가야대학교(고령)_제2캠퍼스,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자연과학계열,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15987,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,변경,폐과,51.6,3.5,54.7,26.7,NaN,NaN,NaN,NaN,NaN,NaN,자연과학계열,30.0,2023
15988,화성의과학대학교,단과대구분없음,컴퓨터학과,폐과,폐과,NaN,NaN,54.7,20.0,NaN,50.0,0.0,0.0,0.0000,0.0,자연과학계열,92.0,2023
15989,화성의과학대학교,단과대구분없음,바이오헬스케어학과,NaN,신설,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.9472,0.0,NaN,NaN,NaN
15990,화성의과학대학교,단과대구분없음,서비스디자인학과,NaN,신설,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0000,0.0,NaN,NaN,NaN


### 계산한 값 병합하기

### 검색 및 엑셀 파일 저장

In [ ]:
#대학원제외파일 엑셀로 저장하기
# df_final_clean_ver.to_excel("0917전체대학데이터셋reset.xlsx", index=False)